# Import Libraries and Data

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
from sklearn.metrics import f1_score, confusion_matrix

# Problem 1a

In [3]:
# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load CIFAR-10 dataset with normalization
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = CNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

# Testing the model
model.eval()
correct = 0
total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Set the model back to training mode
model.train()

# Calculate and print F1 score at the end
f1 = f1_score(all_labels, all_predicted, average='weighted')
print(f'Final F1 Score: {f1:.4f}')

# Calculate and print confusion matrix at the end
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

# After training is complete, print final evaluation accuracy
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 2.243717732941708, Training Time: 11.24 seconds
Epoch 11/300, Loss: 1.501794728476678, Training Time: 15.30 seconds
Epoch 21/300, Loss: 1.314770680864144, Training Time: 16.81 seconds
Epoch 31/300, Loss: 1.1802404245452198, Training Time: 15.08 seconds
Epoch 41/300, Loss: 1.0679627315467581, Training Time: 15.36 seconds
Epoch 51/300, Loss: 0.9713299901741544, Training Time: 15.50 seconds
Epoch 61/300, Loss: 0.884684456888672, Training Time: 15.06 seconds
Epoch 71/300, Loss: 0.8006320481791216, Training Time: 15.60 seconds
Epoch 81/300, Loss: 0.7196936319246316, Training Time: 16.01 seconds
Epoch 91/300, Loss: 0.6396148684994339, Training Time: 17.26 seconds
Epoch 101/300, Loss: 0.5578595217688919, Training Time: 15.91 seconds
Epoch 111/300, Loss: 0.4786670754099136, Training Time: 15.82 seconds
Epoch 121/300, Loss: 0.4006964515732682, Trainin

# Problem 1b

In [3]:
# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load CIFAR-10 dataset with normalization
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the extended CNN model
class ExtendedCNN(nn.Module):
    def __init__(self):
        super(ExtendedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.pool1(self.relu2(self.conv2(x)))
        x = self.pool2(self.relu3(self.conv3(x)))
        x = self.flatten(x)
        x = self.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the extended model
model = ExtendedCNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

# Testing the model
model.eval()
correct = 0
total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Set the model back to training mode
model.train()

# Calculate and print F1 score at the end
f1 = f1_score(all_labels, all_predicted, average='weighted')
print(f'Final F1 Score: {f1:.4f}')

# Calculate and print confusion matrix at the end
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

# After training is complete, print final evaluation accuracy
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 2.280784971878657, Training Time: 11.44 seconds
Epoch 11/300, Loss: 1.5271730456510773, Training Time: 13.70 seconds
Epoch 21/300, Loss: 1.292213365016386, Training Time: 13.82 seconds
Epoch 31/300, Loss: 1.147958550687946, Training Time: 13.74 seconds
Epoch 41/300, Loss: 1.0108811901048627, Training Time: 13.74 seconds
Epoch 51/300, Loss: 0.8800822478121199, Training Time: 13.83 seconds
Epoch 61/300, Loss: 0.7520235096630843, Training Time: 14.10 seconds
Epoch 71/300, Loss: 0.6235002449253941, Training Time: 13.93 seconds
Epoch 81/300, Loss: 0.49445229639177735, Training Time: 14.15 seconds
Epoch 91/300, Loss: 0.36919106499237175, Training Time: 14.40 seconds
Epoch 101/300, Loss: 0.26064762671280395, Training Time: 13.60 seconds
Epoch 111/300, Loss: 0.17583338660962136, Training Time: 13.70 seconds
Epoch 121/300, Loss: 0.11732674452483349, Training Time: